In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from sentence_transformers import SentenceTransformer
import pymysql

In [2]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [12]:
# mysql 연동
conn = pymysql.connect(host='127.0.0.1'
                , port=3306
                , user = 'root'
                , password='root1234'
                , db = 'finaldb'
                , charset='utf8'
                ) 

cur = conn.cursor()

In [18]:
# 플리 id 리스트 만들어서 플리타이틀임베딩 데이터 프레임 생성
cur.execute('SELECT ply_id FROM ply_meta')
lst = cur.fetchall()

ply_id_lst = []
for i in range(len(lst)):
    ply_id_lst.append(lst[i][0])

ply_title_embedding = pd.DataFrame()
ply_title_embedding['ply_id'] = ply_id_lst

In [20]:
# 플리타이틀 임베딩 변환 후 db에 insert
ply_meta_sql = 'SELECT * FROM ply_meta'
ply_meta = pd.read_sql(ply_meta_sql, conn)

ply_title_embedding['ply_title_emd'] = ply_meta.apply(lambda row: model.encode(row.ply_title), axis = 1) # 인코딩 코드

# insert ply_title_embedding  table
for i in range(len(ply_title_embedding)):
    cur.execute(f'INSERT INTO ply_title_embedding (ply_id, ply_title_emd) VALUES \
                ({ply_title_embedding["ply_id"][i]}, "{ply_title_embedding["ply_title_emd"][i]}");')

C:\Users\k9942\AppData\Local\Temp\ipykernel_13072\193671879.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ply_meta = pd.read_sql(ply_meta_sql, conn)


In [21]:
conn.commit()

In [22]:
conn.close()